In [43]:
!python3 -m ensurepip --default-pip
!python3 -m pip install --upgrade pip
!python3 -m pip install requests beautifulsoup4 pandas re random

Looking in links: /var/folders/12/5y1ph4xx3v72zr1wc64m1yrh0000gq/T/tmpfxtg_917
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


# Scraping data from premier league stats to a csv file

In [44]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import random

# Base URL
BASE_URL = "https://fbref.com"

# driver = webdriver.Chrome()
url = "https://fbref.com/en/comps/9/2019-2020/schedule/2019-2020-Premier-League-Scores-and-Fixtures"
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
]

headers = {"User-Agent": random.choice(user_agents)}

time.sleep(random.randint(3, 5))

response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.text, 'html.parser')

matches_table_data = soup.find('table', {'class': 'stats_table'})

matches_tbody_data = matches_table_data.find('tbody')

match_rows_data = matches_tbody_data.find_all('tr')

matches = []

for row in match_rows_data:
    cols = row.find_all('td')

    if len(cols) > 0:
        match_link_tag = cols[11].find('a')
        match_link = f"{BASE_URL}{match_link_tag['href']}" if match_link_tag else None

        hour = cols[2].text.strip()
        date = cols[1].text.strip()
        team_home = cols[3].text.strip()
        xg_home_team = cols[4].text.strip()
        team_away = cols[7].text.strip()
        xg_away_team = cols[6].text.strip()
        referee = cols[10].text.strip()
        score = cols[5].text.strip()

        if date or team_home or team_away or score or match_link:

            home_goals, away_goals = score.split('–')

            if int(home_goals) > int(away_goals):
                result = '1'
            elif int(home_goals) < int(away_goals):
                result = '2'
            else:
                result = 'X'

            matches.append({
                'Date': date,
                'Time': hour,
                'Home Team': team_home,
                'XGH': xg_home_team,
                'Away Team': team_away,
                'XGA': xg_away_team,
                'Score': score,
                'FTHG': home_goals, # full time home goals
                'FTAG': away_goals, # full time away goals
                # 'STH': shots_home,
                # 'STA': shots_away,
                'Result': result,
                'Referee': referee,
                'Match Link': match_link
            })


def fetch_shots_on_target(match):
    try:
        time.sleep(random.randint(1, 5))
        match_page = requests.get(match['Match Link'], headers=headers, timeout=5)
        print(match['Match Link'])
        print(match_page)
        match_soup = BeautifulSoup(match_page.text, 'html.parser')
        match_stats_div = match_soup.find('div', {'id': 'team_stats'})
        match_stats_table = match_stats_div.find('table')

        shots_home, shots_away = None, None

        if match_stats_table:
            match_stats_tbody = match_stats_table.find('tbody')
            match_stats_rows = match_stats_tbody.find_all('tr')
            for row in match_stats_rows:
                cols = row.find_all('td')
                if len(cols) > 0:
                    shots_text_home = cols[0].text.strip()
                    shots_text_away = cols[1].text.strip()

                    shots_home = re.search(r'(\d+)', shots_text_home).group(1) if re.search(r'(\d+)', shots_text_home) else None
                    shots_away = re.search(r'(\d+)', shots_text_away).group(1) if re.search(r'(\d+)', shots_text_away) else None
        match["STH"] = shots_home
        match["STA"] = shots_away
    except Exception as e:
        print(f"Error fetching shots on target: {e}")
        match["STH"] = None
        match["STA"] = None

# for match in matches:
#     fetch_shots_on_target(match)

df = pd.DataFrame(matches)
df.drop(columns=['Match Link'], inplace=True)

df.to_csv('matches-2019-2020.csv', index=False)

print(df.head())

         Date   Time       Home Team  XGH        Away Team  XGA Score FTHG  \
0  2019-08-09  20:00       Liverpool  1.8     Norwich City  0.9   4–1    4   
1  2019-08-10  12:30        West Ham  1.1  Manchester City  3.2   0–5    0   
2  2019-08-10  15:00         Burnley  0.9      Southampton  1.2   3–0    3   
3  2019-08-10  15:00         Watford  0.7         Brighton  0.8   0–3    0   
4  2019-08-10  15:00  Crystal Palace  0.9          Everton  1.1   0–0    0   

  FTAG Result         Referee  
0    1      1  Michael Oliver  
1    5      2       Mike Dean  
2    0      1    Graham Scott  
3    3      2    Craig Pawson  
4    0      X   Jonathan Moss  
